#Setup

In [1]:
!pip install xlsxwriter

     |████████████████████████████████| 148 kB 4.2 MB/s 


In [2]:
import pandas as pd
import numpy as np
import xlsxwriter

import sklearn as sk
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import make_scorer, accuracy_score, cohen_kappa_score, classification_report
from sklearn.model_selection import StratifiedKFold, cross_validate, cross_val_score, GridSearchCV
from sklearn.svm import SVC, SVR, LinearSVC, LinearSVR
from sklearn.linear_model import LogisticRegression, LinearRegression,RidgeClassifier,Ridge
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from matplotlib import pyplot as plt

##Funktionen definieren

In [19]:
#####Modelle erstellen und Bewerten##################
#Custom Metriken definieren
def acc_r (_y_true, _pred):
  result = accuracy_score(_y_true, np.round(_pred,0))
  return result

def kappa_r (_y_true, _pred):
  result = cohen_kappa_score(_y_true, np.clip(np.rint(_pred),0,5), weights='quadratic')
  return result

acc = make_scorer(acc_r)
kappa = make_scorer(kappa_r)

#Funktion um Metriken auszugeben
def get_metrics(_score):
  mean_acc = sum(_score['test_acc'])/splits
  std_acc = _score['test_acc'].std()
  mean_kappa = sum(_score['test_kappa_quad'])/splits
  std_kappa = _score['test_kappa_quad'].std()

  metrics = {
      'mean_acc':mean_acc,
      'std_acc': std_acc,
      'mean_kappa' : mean_kappa, 
      'std_kappa' : std_kappa, 

  }
  return metrics

#Funktion um Modelle zu trainieren (mit Crossvalidation)
def train_model(_classifier, _x, _y,_model_name):
  cv = cross_validate(_classifier,_x,_y, scoring=scoring,
                         cv=folding, return_train_score=False)
  print(cv)
  metrics = get_metrics(cv)
  report = [_model_name,metrics['mean_acc'],metrics['std_acc'],metrics['mean_kappa'],metrics['std_kappa']] 
  return report

#Settings
randomstate = 41
splits = 10
folding = StratifiedKFold(n_splits=splits)
scoring = { 'acc': acc,
            'kappa_quad': kappa,
           #'f1_macro': f1_macro,
           #'f1_micro': f1_micro,
           #'mae_macro': mae_macro,
           }

In [20]:
model = RandomForestClassifier(random_state=randomstate)

#Daten aufbereiten

In [3]:
data = pd.read_excel('/content/drive/MyDrive/Masterarbeit/Data/features.xlsx', index_col=0)  

In [4]:
data.cerf_rating.replace('A1',1.0, inplace=True)
data.cerf_rating.replace(['A2','A2+'],2.0, inplace=True)
data.cerf_rating.replace(['B1','B1+'],3.0, inplace=True)
data.cerf_rating.replace(['B2','B2+'],4.0, inplace=True)
data.cerf_rating.replace(['C1','C2'],5.0, inplace=True)

In [5]:
lables = data['cerf_rating']
all_features = data.drop(columns=['cerf_rating','lex_ttr','lex_verb_var_1','lex_sqrt_verb_var_1','lex_cor_verb_var_1','lex_wiener_sachtextformel','morph_first_person_fin_verb_ratio'])

In [40]:
#Featureset mit niedriger korrelation (-0.5 >= r <= 0.5) erstellen
corr = all_features.drop(columns=['shal_text_length']).corrwith(all_features['shal_text_length'],axis=0,method='pearson',)
all_features = all_features.drop(columns=['shal_text_length'])
low_corr = []
for index in corr.index:
  if corr[index] >= -0.5 and corr[index] <= 0.5:
    low_corr.append(index)

low_corr_features = all_features[low_corr]

KeyError: ignored

In [11]:
lex = []
syn = []
shal = []
morph = []
for name in low_corr:
  if 'lex_' in name:
    lex.append(name)
  elif 'shal_' in name or 'errors' in name:
    shal.append(name)
  elif 'morph_' in name:
    morph.append(name)
  elif 'syn_' in name or 'dep_' in name:
    syn.append(name)
lex_features = low_corr_features[lex]
syn_features = low_corr_features[syn]
shal_features = low_corr_features[shal]
morph_features = low_corr_features[morph]

In [8]:
features = [lex_features,syn_features,shal_features,morph_features]

In [41]:
#Features normalisieren [0,1]
scaler = MinMaxScaler(feature_range=(0,1))
input_33 = scaler.fit_transform(low_corr_features)
input_43 = scaler.fit_transform(all_features)

#Feature mit höchster Vorhersagekraft

In [21]:
model = RandomForestClassifier(random_state=randomstate)

In [49]:
mapping_names = {'lex_hdd':'HD-D',
                 'lex_mtld':'MTLD',
                 'lex_adverb_var':'Adverb Variation',
                 'lex_modifier_var':'Adjektiv Variation',
                 'lex_noun_var': 'Nomen Variation',
                 'lex_verb_var_2':'Verb Variation',
                 'freq_avg_log_word_frequency': 'Durchschn. logarithmische Worthäufigkeit',
                 'er_avg_num_errors':'Durschn. Anzahl an Rechtschreibfehlern',
                 'dep_per_noun':'Durchschn. Anzahl an Abhängigkeitsbeziehungen pro Nomen',
                 'dep_per_verb':'Durchschn. Anzahl an Abhängigkeitsbeziehungen pro Verb',
                 'syn_avg_len_clause':'Durchschn. Länge der Teilsätze',
                 'syn_avg_nonterminals_per_sent':'Durchschn. Anteil an Nebensätzen',
                 'syn_avg_nonterminals_per_sent':'Durchschn. Anzahl an Nichtterminalsymbolen',
                 'syn_avg_tunit_length':'Durchschn. T-Unit',
                 'syn_avg_vz_per_sent':'Durchschn. Anzahl an Infinitivsätzen mit zu',
                 'syn_avg_corinate_phrases_per_sent':'Durchschn. Anteil an Nebengeordneter Sätze',
                 'syn_dep_clause_ratio':'Durchschn. Anteil an Nebensätzen',
                 'syn_dep_clause_w_conj_to_dep_clause':'Durchschn. Anteil an Nebensätzen mit Konjunktion',
                 'syn_avg_vp_length':'Durchschn. Länge der Verbalphrasen',
                 'lex_yules_k':'Yules K',
                 'lex_uber_index':'Uber Index',
                 'lex_root_ttr': 'root TTR',
                 'lex_fog': 'FOG-Index',
                 'lex_smog': 'SMOG-Index',
                 'shal_chars_per_word':'Durchschn. Wortlänge',
                 'shal_syl_per_word':'Durchschn. Anzahl an Silben pro Wort',
                 'shal_polysyl_to_word':'Rel. Anteil an mehrsilbigen Wörtern',
                 'shal_sents_length':'Durchschn. Satzlänge',
                 'morph_comp_noun_ratio':'Rel. Anteil zusammengesetzter Nomen',
                 'morph_first_person_verb_ratio':'Rel. Anteil an Verben in der ersten Person',
                 'morph_second_person_verb_ratio':'Rel. Anteil an Verben in der zweiten Person',
                 'morph_third_person_verb_ratio':'Rel. Anteil an Verben in der dritten Person',
                 'morph_first_person_fin_verb_ratio': 'Rel. Anteil an finiten Verben',
                 'morph_past_tense_verb_ratio':'Rel. Anteil der Verben in der Vergangenheitsform',
                 'morph_past_participle_verb_ratio':'Rel. Anteil an Verben im Pratizip Perfekt',
                 'morph_nominative_nouns_ratio':'Rel. Anteil von Nomen im Nominativ',
                 'morph_genitiv_nouns_ratio':'Rel. Anteil von Nomen im Genitiv',
                 'morph_dativ_nouns_ratio':'Rel. Anteil von Nomen im Dativ',
                 'morph_singular_nouns_ratio': 'Rel. Anteil von Nomen im Singular',
                 'morph_plural_nouns_ratio':'Rel. Anteil von Nomen im Plural',
                 'morph_akkusativ_nouns_ratio':'Rel. Anteil von Nomen im Akkusativ',
                 'morph_keit_suffix_ratio':'Rel. Anteil der Nomen mit der Endung -keit',
                 'morph_ung_suffix_ratio':'Rel. Anteil der Nomen mit der Endung -ung',
                 'morph_werk_suffix_ratio':'Rel. Anteil der Nomen mit der Endung -werk',
                 'morph_article_ratio':'Rel. Anteil an Artikeln',
}

In [55]:
def f_importances(coef, names, top=-1):
    imp = coef
    imp, names = zip(*sorted(list(zip(imp, names))))
    d = {'Feature': names[::-1][0:top],
         'Wichtigkeit': imp[::-1][0:top]}
    df = pd.DataFrame(d)
    return df

##reduziertes Feature-Set

In [53]:
feature_names_33 = []
for feature in low_corr_features.columns:
  if mapping_names[feature] not in feature_names_33:
    feature_names_33.append(mapping_names[feature])

In [54]:
model.fit(input_33, lables)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=41, verbose=0,
                       warm_start=False)

In [56]:
importance = f_importances(abs(model.feature_importances_), feature_names, top=10)

In [57]:
importance

,Feature,Wichtigkeit
0,Durchschn. Wortlänge,0.052487
1,Nomen Variation,0.052458
2,Durchschn. Anzahl an Nichtterminalsymbolen,0.052044
3,Durchschn. T-Unit,0.051916
4,Durchschn. Anzahl an Silben pro Wort,0.050222
5,Rel. Anteil an Verben in der zweiten Person,0.049258
6,HD-D,0.040774
7,Rel. Anteil von Nomen im Nominativ,0.040045
8,Rel. Anteil an Verben in der ersten Person,0.037431
9,Durschn. Anzahl an Rechtschreibfehlern,0.035733


In [58]:
importance.to_excel('/content/drive/MyDrive/Masterarbeit/Data/1_2_klassisch_feature_wichtigkeit_33.xlsx', engine='xlsxwriter', index=False)  

##gesamtes Feature-Set

In [59]:
feature_names_43 = []
for feature in all_features.columns:
  if mapping_names[feature] not in feature_names_43:
    feature_names_43.append(mapping_names[feature])

In [60]:
model.fit(input_43, lables)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=41, verbose=0,
                       warm_start=False)

In [61]:
importance = f_importances(abs(model.feature_importances_), feature_names, top=10)

In [62]:
importance

,Feature,Wichtigkeit
0,Adjektiv Variation,0.095889
1,Rel. Anteil von Nomen im Dativ,0.063895
2,Rel. Anteil von Nomen im Genitiv,0.059732
3,Rel. Anteil an Verben im Pratizip Perfekt,0.049363
4,HD-D,0.039532
5,Rel. Anteil der Verben in der Vergangenheitsform,0.034165
6,Durschn. Anzahl an Rechtschreibfehlern,0.030700
7,Durchschn. Anzahl an Abhängigkeitsbeziehungen ...,0.028910
8,MTLD,0.028118
9,Rel. Anteil an Verben in der dritten Person,0.027155


In [63]:
importance.to_excel('/content/drive/MyDrive/Masterarbeit/Data/1_2_klassisch_feature_wichtigkeit_43.xlsx', engine='xlsxwriter', index=False)  

#t-test